In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_predict, KFold

%matplotlib inline

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_sub = pd.read_csv("data/sample_submission.csv")

print (train.shape)

(4459, 4993)


In [18]:
X_train_base = train.drop(['ID','target'], axis=1)
X_test_base = test.drop('ID', axis=1)
y_base = np.log1p(train['target'].values)

In [4]:
import lightgbm as lgb
kf = KFold(n_splits=10, shuffle=True, random_state=42)
model = lgb.LGBMRegressor(objective='regression',
                         num_leaves=31,
                         n_estimators=500)

In [ ]:
predict = cross_val_predict(model, X_train_base, y_base, cv=kf)

In [23]:
def prepare(data_orig):
    data = pd.DataFrame()
    data['mean'] = data_orig.mean(axis=1)
    data['std'] = data_orig.std(axis=1)
    data['min'] = data_orig.min(axis=1)
    data['max'] = data_orig.max(axis=1)
    data['range'] = data_orig.max(axis=1) - data_orig.min(axis=1)
    data['coef_of_var'] = data_orig.apply(lambda row: np.var(row)/np.mean(row), axis=1)
    data['number_of_different'] = data_orig.nunique(axis=1)               # Number of diferent values in a row.
    data['non_zero_count'] = data_orig.fillna(0).astype(bool).sum(axis=1)# Number of non zero values (e.g. transaction count)
    #data['sum'] = data_orig.sum(axis=1)
    
    #data['cumsum'] = data_orig.cumsum()
    return data

# Replace 0 with NaN to ignore them.
X_test = prepare(X_test_base.replace(0, np.nan))
X_train = prepare(X_train_base.replace(0, np.nan))

In [24]:
model.fit(X_train, y_base)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=500,
       n_jobs=-1, num_leaves=31, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [26]:
y_pred = model.predict(X_test)
sample_sub.target = np.exp(y_pred)-1
sample_sub.to_csv("own_feat_sub.csv", index=False)

In [22]:
sample_sub.shape

(49342, 2)

In [21]:
X_test_base.shape

(49342, 4991)